In [1]:
# Load the modules and be done with them
import theano as th
from keras.models import Sequential
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.layers.convolutional import Conv1D
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn import preprocessing

Using TensorFlow backend.


In [10]:
#Load the data from the WISDM dataset
dataframe = pd.read_csv("WISDM_ar_v1_1_raw.csv", header=None, 
                       names = ["_id", "activity", "timestamp", "acceleration_x", "acceleration_y", "acceleration_z"])
dataframe = dataframe[["timestamp", "acceleration_x", "acceleration_y", "acceleration_z", "activity"]]
dataframe.set_index("timestamp")


/Users/gafergus/anaconda/envs/tensorflow/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,acceleration_x,acceleration_y,acceleration_z,activity
timestamp,,,,
49105962326000,-0.694638,12.680544,0.503953,Jogging
49106062271000,5.012288,11.264028,0.953424,Jogging
49106112167000,4.903325,10.882658,-0.0817221,Jogging
49106222305000,-0.612916,18.496431,3.02372,Jogging
49106332290000,-1.184970,12.108489,7.20516,Jogging
49106442306000,1.375655,-2.492524,-6.51053,Jogging
49106542312000,-0.612916,10.569390,5.70693,Jogging
49106652389000,-0.503953,13.947236,7.05534,Jogging
49106762313000,-8.430995,11.413852,5.13487,Jogging


In [12]:
np.random.seed(102849201)

In [14]:
# load dataset
dataset = dataframe.values
X = dataset[:,0:3].astype(float)
X = preprocessing.scale(X)
Y = dataset[:,4]
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
#convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [8]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(9, input_dim=3, init='normal', activation='relu'))
    model.add(Dense(3, init='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
Conv1D(1, 300)

In [17]:
model = Sequential()
#model.add(Dense(300, input_dim=3, init='normal', activation='sigmoid'))
model.add(Conv1D(100, 20, activation='sigmoid', input_dim=3))
#model.add(MaxPooling1D(pool_length=4)) 
model.add(Dense(100, init='normal', activation='sigmoid'))
model.add(Dense(50, init='normal', activation='sigmoid'))
model.add(Dense(6, init='normal', activation='sigmoid'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, dummy_y, batch_size=1000)

ValueError: Error when checking model input: expected convolution1d_input_4 to have 3 dimensions, but got array with shape (1098204, 3)

In [67]:
#model.evaluate(X, dummy_y, batch_size=5)
#model.metrics_names 

In [68]:
#estimator = KerasClassifier(build_fn=baseline_model, nb_epoch=200, batch_size=5, verbose=0)

In [69]:
#kfold = KFold(n_splits=10, shuffle=True, random_state=73472991)

In [71]:
#results = cross_val_score(estimator, X, dummy_y, cv=kfold)
#print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))